In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, CSVSearchTool
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

/Users/td/.pyenv/versions/3.12.0/lib/python3.12/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/td/.pyenv/versions/3.12.0/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/Users/td/.pyenv/versions/3.12.0/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipVal

<div style="text-align: center;">
<!--     <h1 style="color: purple; font-size: 3.5em;">MMA AI</h1> -->
<!--     <br> -->
    <img src="./images/holloway.jpeg" alt="Holloway" style="max-width: 70%; height: auto;">
<!--     <br> -->
    <h1 style="color: purple; font-size: 3.5em;">MMA AI</h1>
</div>


In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["OPENAI_MODEL_NAME"]="gpt-4o"

In [4]:
search_tool = SerperDevTool()

#csv_tool1 = CSVSearchTool(csv='./data/master.csv')
csv_tool1 = CSVSearchTool(csv='./data/fighter_info.csv')
csv_tool2 = CSVSearchTool(csv='./data/event_data_sherdog.csv')
# directory_tool = DirectoryReadTool(directory='./data')

In [5]:
# fighter1 = input("Fighter 1: ")
# fighter2 = input("Fighter 2: ")

# fighter1 = 'Michael Page'
# fighter2 = 'Ian Garry'

#fighter1 = 'Brian Ortega'
#fighter2 = 'Diego Lopes'

fighter1 = 'Gilbert Burns'
fighter2 = 'Michael Morales'


In [6]:
agent1 = Agent(
    role='Fight Researcher & Analyst',
    goal='Gather comprehensive information about upcoming UFC fights',
    backstory="""
    You are a seasoned analyst with a deep understanding of MMA/UFC.
    You specialize in breaking down fights, with expertise in dissecting historical fight outcomes and statistics.
    You provide insights into how a fighter's recent experiences might influence their next performance.
    """,
    tools=[csv_tool1, csv_tool2],
#     allow_delegation=True
)

agent2 = Agent(
    role='Public Sentiment Researcher & Analyst',
    goal='Analyze public bias, sentiment, and unusual news related to the fighters',
    backstory="""
    You are an expert in public sentiment analysis.
    You specialize in gathering and interpreting data from news sources, social media, and other public platforms.
    Your keen insight helps uncover public biases, general sentiment, and any unusual news that might impact the perception of fighters.
    """,
    tools=[search_tool],
    allow_delegation=True
)

agent_final = Agent(
    role='Expert MMA Handicapper',
    goal='Generate detailed predictions for upcoming UFC fights based on gathered data and professional techniques',
    backstory="""
    You are a professional handicapper in Las Vegas.
    You are an expert in predictive analytics and fight analysis.
    You integrate detailed statistics, historical performances, and public sentiment to provide a comprehensive prediction for upcoming fights.
    Your insights help forecast the potential outcome with high accuracy.""",
    tools=[search_tool],
    allow_delegation=True
)


In [7]:
task1 = Task(
    description=f"Research detailed statistics and historical performance of {fighter1}",
    expected_output=f"Comprehensive statistical report on {fighter1}.",
    agent=agent1,
    output_file=f"{fighter1}_vs_{fighter2}/upcoming_fight_research_{fighter1}.md"
)

task2 = Task(
    description=f"Research detailed statistics and historical performance of {fighter2}",
    expected_output=f"Comprehensive statistical report on {fighter2}.",
    agent=agent1,
    output_file=f"{fighter1}_vs_{fighter2}/upcoming_fight_research_{fighter2}.md"
)

task3 = Task(
    description=f"Analyze public sentiment, bias, and unusual news about the upcoming fight between {fighter1} and {fighter2}.",
    expected_output=f"Sentiment analysis report on public perception, bias, and unusual news regarding the fight between {fighter1} and {fighter2}.",
    agent=agent2,
    output_file=f"{fighter1}_vs_{fighter2}/upcoming_fight_research_sentiment_{fighter1}_vs_{fighter2}.md"
)

task4 = Task(
    description=f"Generate American betting lines for {fighter1} vs {fighter2}.",
    expected_output="American betting lines for the fight, including moneyline odds for both fighters and method of victory.",
    agent=agent_final,
    output_file=f"{fighter1}_vs_{fighter2}/upcoming_fight_research_odds_{fighter1}_vs_{fighter2}.md"
)

task_final = Task(
    description=f"Generate a detailed prediction for the upcoming fight between {fighter1} and {fighter2} with a detailed explanation of your reasoning behind it using all gathered information",
    expected_output="A prediction of the fight outcome with a detailed explanation of your reasoning behind it.",
    agent=agent_final,
    output_file=f"{fighter1}_vs_{fighter2}/upcoming_fight_prediction_{fighter1}_vs_{fighter2}.md"
)

os.makedirs(f"{fighter1}_vs_{fighter2}", exist_ok=True)

In [9]:
crew = Crew(
    agents=[agent1, agent2, agent_final],
    tasks=[task1, task2, task3, task4, task_final],
    #verbose=2
)

result = crew.kickoff()
print(result)

2025-05-16 02:28:45,964 - 8632344320 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of existing embedding ID: default-app-id--d71ce093fa39760341b006ba5c79e541aab8b4a4d38ef05c0d1c7ae0f603410e


The upcoming fight between Gilbert Burns and Michael Morales promises to be a compelling clash of styles. Gilbert Burns, known for his exceptional grappling skills, has a proven track record in securing victories through submissions. His recent performances have shown his ability to endure through rounds and capitalize on precise moments to execute submission holds, making him a dangerous competitor when the fight goes to the ground.

On the other hand, Michael Morales comes into this fight with an untarnished record of 17-0. His knockout power is a significant factor, having achieved 12 of his 17 wins by knockout. Morales's proficiency in striking, combined with his height advantage at 6 feet, positions him well to manage distance and execute powerful strikes aimed at neutralizing Burns's takedown attempts.

Public sentiment heavily favors Morales due to his undefeated status and knockout ratio, creating an image of an emerging star rising against a seasoned veteran. The narrative sur